In [ ]:
# setup: 
import csv #for CSV handling
import re #for regular expressions handling

In [ ]:
# check whether columns have the right title, if so, name them for easier handling 
def set_column_name(column_name, column_number, column_title):
    if header[column_number] != column_title:
        print('Error at', column_name,  'column number')
        sys.exit(1) # evtl Luft nach oben bei dem Befehl hier
    else: column_name = column_number

For 5.5: Process KATI data on Internationality, Step 1: calculate for each year and each number of countries how many single/multiple author papers there are with this number of countries involved

In [ ]:
source_file_name = '.csv'  #input are "Internationality" files which have been brought to CSV format without " manually
with open(source_file_name, 'r') as source_file:
    reader = csv.reader(source_file, delimiter = ';')
    header = next(reader)
    print(header)
    set_column_name(year_column, 1, 'year')
    set_column_name(number_countries_column, 4, 'countryCount')
    set_column_name(number_authors_column, 2, 'Nauthor')
    data = {}     #sort data by year and make it accessible by year
    for row in reader:
        year = row[year_column]
        if year not in data:
            data[year] = [row]
        else: 
            data[year].append(row)
    with open(source_file_name.rstrip('.csv')+'_country_numbers_raw.csv', 'w', newline='') as target_file:
        writer = csv.writer(target_file)
        header = ['year', '#countries', 'document count single author', 'document count multiple authors']
        writer.writerow(header)
        for year in data:
            max_num_countries = max(int(row[number_countries_column]) for row in data[year])
            print (year, '   ', max_num_countries) #for plausibility check while running
            for this_num_countries in range(0,max_num_countries+1): 
                counter_this_num_countries_single_author = 0 #count single-author papers seperately
                counter_this_num_countries_multiple_authors = 0
                for row in data[year]:
                    if int(row[number_countries_column]) == this_num_countries:
                        if int(row[number_authors_column]) == 1:
                            counter_this_num_countries_single_author += 1 
                        else:
                            counter_this_num_countries_multiple_authors += 1
                writer.writerow([year, this_num_countries, counter_this_num_countries_single_author, counter_this_num_countries_multiple_authors])
                
print('done')

For 5.5.: Step 2: aggregate into clusters of numbers and return maximal number of countries per year

In [ ]:
source_file_name = ''  #input are "Internationality" files which have been brought to CSV format without " manually
with open(source_file_name, 'r') as source_file:
    reader = csv.reader(source_file, delimiter = ';')
    header = next(reader)
    print(header)
    set_column_name(year_column, 1, 'year')
    set_column_name(number_countries_column, 4, 'countryCount')
    set_column_name(number_authors_column, 2, 'Nauthor')
    data = {}
    for row in reader:
        year = row[year_column]
        if year not in data:
            data[year] = [row]
        else: 
            data[year].append(row)
    with open(source_file_name.rstrip('.csv')+'_country_numbers_aggr.csv', 'w', newline='') as target_file:
        writer = csv.writer(target_file)
        header = ['year', '#:0country', '#:1country1author', '#:1country_more_authors', '#:2countries', '#:3-5countries', '#:6-10countries','#:>=11countries', 'max # of countries']
        writer.writerow(header)
        for year in data:
            max_num_countries = max(int(row[number_countries_column]) for row in data[year])
            print (year, '   ', max_num_countries)
            counter_no_country = 0
            counter_1_country_1_author = 0
            counter_1_country_more_authors = 0
            counter_2_countries = 0
            counter_3_to_5_countries = 0
            counter_6_to_10_countries = 0
            counter_many_countries = 0
            for this_num_countries in range(0,max_num_countries+1):
                counter_this_num_countries_single_author = 0
                counter_this_num_countries_multiple_authors = 0
                temp_count_3_to_5 = 0
                temp_count_6_to_10 = 0
                temp_count_many = 0
                for row in data[year]:
                    if int(row[number_countries_column]) == this_num_countries:
                            if int(row[number_authors_column]) == 1:
                                counter_this_num_countries_single_author += 1 
                            else:
                                counter_this_num_countries_multiple_authors += 1
                if this_num_countries == 0:
                   counter_no_country = counter_this_num_countries_single_author + counter_this_num_countries_multiple_authors
                else:
                    if this_num_countries == 1:
                       counter_1_country_1_author = counter_this_num_countries_single_author
                       counter_1_country_more_authors = counter_this_num_countries_multiple_authors
                    else:
                        if this_num_countries == 2:
                            counter_2_countries = counter_this_num_countries_single_author + counter_this_num_countries_multiple_authors
                        else:
                            if 3 <= this_num_countries <= 5:
                               temp_count_3_to_5 = counter_this_num_countries_single_author + counter_this_num_countries_multiple_authors
                            else:
                                if 6 <= this_num_countries <= 10:
                                   counter_6_to_10_countries = counter_this_num_countries_single_author + counter_this_num_countries_multiple_authors
                                else:
                                     temp_count_many = counter_this_num_countries_single_author + counter_this_num_countries_multiple_authors
                counter_3_to_5_countries += temp_count_3_to_5 
                counter_6_to_10_countries += temp_count_6_to_10
                counter_many_countries += temp_count_many
            writer.writerow([year, counter_no_country,counter_1_country_1_author, counter_1_country_more_authors, counter_2_countries, counter_3_to_5_countries, counter_6_to_10_countries, counter_many_countries, max_num_countries])
    
print('done')
#final grouping into single author, national, international was done directly in Excel

For 5.2: Preprocess country data by clustering by periods; check for country name issues

In [ ]:
source_file_name = 'Countries_List.csv'  
with open(source_file_name, 'r') as source_file:
    reader = csv.reader(source_file, delimiter = '"') #use " as delimiter to avoid trouble with the commata in the names
    header = next(reader)
    print(header)
    set_column_name(year_column, 1, 'year')
    set_column_name(country_name_column, 3, 'countryLabel')
    set_column_name(country_tag_column, 5, 'iso3')
    set_column_name(pub_num_column, 7, 'N_pub')
    set_column_name(continent_column, 11, 'continentLabel')
    data = {}
    for row in reader:
        country_tag = row[country_tag_column]
        if country_tag not in data:
            data[country_tag] = [row]
        else: 
            data[country_tag].append(row)
    with open(source_file_name.rstrip('.csv')+'_aggr_periods.csv', 'w', newline='') as target_file:
        writer = csv.writer(target_file)
        header = ['Continent', 'Country Name', '#Publications 1970-1999', '#Publications 2000-2009', '#Publications 2010-2020', '#Publications 2020-2025']
        writer.writerow(header)
        for iso3 in data:
            counter_to_1999 = 0
            counter_2000_2009 = 0
            counter_2010_2019 = 0
            counter_from_2020 = 0
            countryLabel = ''
            continent = ''
            for row in data[iso3]:
                if countryLabel != str(row[country_name_column]) and countryLabel != '':
                    print(iso3, ' has two names: ', countryLabel, ' and ', str(row[country_name_column])) #check if country names are ambiguous
                if int(row[year_column]) <= 1999:
                    counter_to_1999 += int(row[pub_num_column])
                else:
                    if 2000 <= int(row[year_column]) <= 2009:
                        counter_2000_2009 += int(row[pub_num_column])
                    else:
                        if 2010 <= int(row[year_column]) <= 2019:
                            counter_2010_2019 += int(row[pub_num_column])
                        else: 
                             counter_from_2020 += int(row[pub_num_column])
                countryLabel = str(row[country_name_column])
                if str(row[continent_column]).endswith('america'):
                    continent =  str(row[continent_column]).rstrip('america').capitalize() + ' America'
                else: 
                    continent = str(row[continent_column]).capitalize()
            writer.writerow([continent, countryLabel,counter_to_1999, counter_2000_2009, counter_2010_2019, counter_from_2020 ])
  

print('done')

For 5.7 : Preprocess Country Data by collect DACH, UK, US data per year; as well as total number per year

In [ ]:
source_file_name = 'Countries_List.csv'  
with open(source_file_name, 'r') as source_file:
    reader = csv.reader(source_file, delimiter = '"') #use " as delimiter to avoid trouble with the commata in the names
    header = next(reader)
    print(header)
    set_column_name(year_column, 1, 'year')
    set_column_name(country_name_column, 3, 'countryLabel')
    set_column_name(country_tag_column, 5, 'iso3')
    set_column_name(pub_num_column, 7, 'N_pub')
    set_column_name(continent_column, 11, 'continentLabel')
    data = {}
    for row in reader:
        year = row[year_column]
        if year not in data:
            data[year] = [row]
        else: 
            data[year].append(row)
    with open(source_file_name.rstrip('.csv')+'_DACH_UK_US_all.csv', 'w', newline='') as target_file:
        writer = csv.writer(target_file)
        header = ['Year', '#Pubs DACH', '#Pubs UK', '#Pubs US', '# all Pubs']
        writer.writerow(header)
        for year in data:
            counter_DACH = 0
            counter_UK = 0
            counter_US = 0
            counter_all = 0
            for row in data[year]:
                iso3 = row[country_tag_column]
                counter_all += int(row[pub_num_column])
                if iso3 == 'deu' or iso3 == 'aut' or iso3 == 'che':
                    counter_DACH += int(row[pub_num_column])
                if iso3 == 'gbr':
                    counter_UK += int(row[pub_num_column])
                if iso3 == 'usa':
                    counter_US += int(row[pub_num_column])
 
            #print(iso3, ' ', counter_to_1999, counter_2000_2009, counter_2010_2019, counter_from_2020)
            writer.writerow([year, counter_DACH, counter_UK, counter_US, counter_all])
print('done')

For 5.4: Clean name data in co-publication files by bringing all names to 'Last name, Initial' format:

In [ ]:
source_file_name = 'Koauthor List Fullerene.csv'  
with open(source_file_name, 'r') as source_file:
    reader = csv.reader(source_file, delimiter = '"') # use " (around all strings in the source file) as the read-delimiter to avoid trouble with the commata in the names
    header = next(reader)
    print(header)
    set_column_name(ID_column, 1, 'uid')
    set_column_name(year_column, 3, 'year')
    set_column_name(names_column, 5, 'author_list')
    with open(source_file_name.rstrip('.csv')+'_clean_names.csv', 'w', newline='') as target_file: # "intermediate" file for QA purposes
            writer = csv.writer(target_file)
            header = ['ID', 'year', 'authors']
            writer.writerow(header)
            for row in reader: 
                #if len(row) >= 6: 
                    authors_list = []
                    name = str(row[names_column]).title()
                    # bring all names to format with capitalised first letter, rest in lower case
                    names_list = name.split('|')
                    # split at '|‘ (delimiter between different author names) 
                    for name in names_list:
                        name_split = [string.strip() for string in name.split(',')]
                        # split each name at comma between last name and first names, and remove all leading/trailing white spaces
                        if len(name_split) > 1: 
                            name_clean = name_split[0] + ', ' + name_split[1][0]
                        else: 
                            name_clean = name_split[0] + ', ' 
                        # cleaned name is of format [last name], [first initial], unless no info about first name(s) is given; name format in this case is [last name], [blank]
                        if name_clean not in authors_list:
                            authors_list.append(name_clean)
                            # add cleaned name to authors' list unless it is already there
                    ID = row[ID_column]
                    year = row[year_column]
                    writer.writerow([ID, year, authors_list])
print('Done')
# POTENTIAL IMPROVEMENT: catch different spellings of last names (Müller vs Mueller etc.)
   
    
   

For 5.4: Create list of co-publication graph edges 

In [ ]:
source_file_name = 'Koauthor List Fullerene_clean_names.csv'  
with open(source_file_name, 'r') as source_file:
    reader = csv.reader(source_file, delimiter = ',') 
    header = next(reader)
    print(header)
    set_column_name(authors_column, 2, 'authors')
    set_column_name(year_column, 1, 'year')
    next_author_id = 0 
    author_ids = {}
    def author_id(author):
        global next_author_id
        global author_ids
       # print(next_author_id)
        if author not in author_ids:
            author_ids[author] = next_author_id
            next_author_id += 1
        return author_ids[author]
        
    with open(source_file_name.rstrip('clean_names.csv')+'_pub_graph.csv', 'w', newline='') as target_file: 
            writer = csv.writer(target_file)
            header = ['author ID 1', 'author ID 2', 'author 1', 'author 2', 'year']
            writer.writerow(header)   
            for row in reader:
                row_clean = row[authors_column].strip("[']")
                authors = row_clean.split("', '")
                year = row[year_column]
                for i in range(len(authors)):
                    for j in range(i, len(authors)):
                        writer.writerow([author_id(authors[i]), author_id(authors[j]), authors[i], authors[j], year])

    print('done')
    #last step: calculating components, and the share of the biggest component, is done with connected.py


For 5.6: Sort papers by methods:
Step 1: consolidate Abstracts and Auto Keywords

In [ ]:
source_file_name = 'Titel Abtract Countries_fixed.csv'
keyword_file_name_1 = 'Auto Keywords.csv'
with open(source_file_name, 'r') as source_file: #master file for data 
    reader = csv.reader(source_file, delimiter = ',') 
    header = next(reader)
    print(header)
    set_column_name(title_column, 0, 'title')
    set_column_name(uid_column, 3, 'uid')
    set_column_name(year_column, 6, 'year')
    set_column_name(abstract_column, 9, 'abstract')
    set_column_name(country_count_column, 12, 'countryCount')
    set_column_name(countries_column, 15, 'countryList')
    with open(keyword_file_name_1, 'r') as source_file_KW: #additional file with keywords
        reader2 = csv.reader(source_file_KW, delimiter = '"') 
        header2 = next(reader2)
        print(header2)
        KW_uid_column = 1
        if header2[KW_uid_column] != 'uid':
            print('Error at "KW publication ID" column number')
            sys.exit # evtl Luft nach oben bei dem Befehl hier
        KW_year_column = 3
        if header2[KW_year_column] != 'year':
            print('Error at "KW year" column number')
            sys.exit # evtl Luft nach oben bei dem Befehl hier
        KW_tags_column = 5
        if header2[KW_tags_column] != 'autotagList':
            print('Error at "KW tags" column number')
            sys.exit # evtl Luft nach oben bei dem Befehl hier
        data_KW = {}
        for row in reader2:
            uid_KW = row[KW_uid_column]
            if uid_KW not in data_KW:
                data_KW[uid_KW] = [row]
            else: 
                data_KW[uid_KW].append(row) #KW data sorted by uid
        with open(source_file_name.rstrip('.csv')+'_KW.csv', 'w', newline='') as target_file: #new write file for consolidated data
            writer = csv.writer(target_file)
            header_w = ['UID', 'Title', 'Year', 'Abstract', 'Auto Keywords', 'Country Count', 'Country List']
            writer.writerow(header_w)
            for row in reader: #take each article in the master data set
                year = row[year_column]
                uid = row[uid_column]
                title = row[title_column]
                abstract = row[abstract_column]
                country_count = row[country_count_column]
                countries = row[countries_column]
                if uid in data_KW:
                    for row in data_KW[uid]: #find same uid in KW data set
                        if row[KW_year_column] != year: #check for inconsistencies
                            print('Years not matching for UID', uid)
                            sys.exit # evtl Luft nach oben bei dem Befehl hier
                        if len(row) >=6 and len(row[KW_tags_column]) > 0:
                            KW_list = row[KW_tags_column]#.split('|') #if keywords exist, split the string into single keywords
                            writer.writerow([uid, title, year, abstract, KW_list, country_count, countries])
                        else:
                           writer.writerow([uid, title, year, abstract, '', country_count, countries]) 
                else:
                    writer.writerow([uid, title, year, abstract, '', country_count, countries])
                    print('no UID match in auto KW data for UID', uid)
print('Done')
                    
            
                
                
        
        

For 5.6 Step 2:
Consolidate Abstracts, Auto KWs, and Manual KWs:

In [ ]:
source_file_name = 'Titel Abtract Countries_fixed_KW.csv'
keyword_file_name = 'Manual Keywords.csv'
with open(source_file_name, 'r') as source_file: #master file for data 
    reader = csv.reader(source_file, delimiter = ',') 
    header = next(reader)
    print(header)
    set_column_name(title_column, 1, 'Title')
    set_column_name(uid_column, 0, 'UID')
    set_column_name(year_column, 2, 'Year')
    set_column_name(abstract_column, 3, 'Abstract')
    set_column_name(auto_KW_column, 4, 'Auto Keywords')
    set_column_name(country_count_column, 5, 'Country Count')
    set_column_name(countries_column, 6, 'Country List')
    with open(keyword_file_name, 'r') as source_file_KW: #additional file with keywords
        reader2 = csv.reader(source_file_KW, delimiter = '"') 
        header2 = next(reader2)
        print(header2)
        set_column_name(KW_uid_column, 1, 'uid')
        set_column_name(KW_year_column, 3, 'year')
        set_column_name(KW_tags_column, 5, 'manualtagList')
        data_KW = {}
        for row in reader2:
            uid_KW = row[KW_uid_column]
            if uid_KW not in data_KW:
                data_KW[uid_KW] = [row]
            else: 
                data_KW[uid_KW].append(row) #KW data sorted by uid
        with open(source_file_name.rstrip('.csv')+'_all.csv', 'w', newline='') as target_file: #new write file for consolidated data
            writer = csv.writer(target_file)
            header_w = ['UID', 'Title', 'Year', 'Abstract', 'Auto Keywords', 'Manual Keywords','Country Count', 'Country List']
            writer.writerow(header_w)
            for row in reader: #take each article in the master data set
                year = row[year_column]
                uid = row[uid_column]
                title = row[title_column]
                abstract = row[abstract_column]
                auto_KW_list = row[auto_KW_column]
                country_count = row[country_count_column]
                countries = row[countries_column]
                if uid in data_KW:
                    for row in data_KW[uid]: #find same uid in KW data set
                        if row[KW_year_column] != year:  #check for inconsistencies
                            print('Years not matching for UID', uid)
                            sys.exit # evtl Luft nach oben bei dem Befehl hier
                        if len(row) >=6 and len(row[KW_tags_column]) > 0:
                            man_KW_list = row[KW_tags_column]#.split('|') #if keywords exist, split the string into single keywords
                            writer.writerow([uid, title, year, abstract, auto_KW_list, man_KW_list, country_count, countries])
                        else:
                           writer.writerow([uid, title, year, abstract,auto_KW_list, '', country_count, countries]) 
                else:
                    writer.writerow([uid, title, year, abstract, auto_KW_list,'', country_count, countries])
                    print('no UID match in manual KW data for UID', uid)
print('Done')
                    
            
                
                
        
        

For 5.6 Step 3:
Search all three fields for mentions of certain methods 

In [ ]:
source_file_name = 'Titel Abtract Countries_fixed_KW_all.csv'
with open(source_file_name, 'r') as source_file: 
    reader = csv.reader(source_file, delimiter = ',') 
    header = next(reader)
    prsource_file_name = 'Titel Abtract Countries_fixed_KW_all.csv'
with open(source_file_name, 'r') as source_file: 
    reader = csv.reader(source_file, delimiter = ',') 
    header = next(reader)
    print(header)
    set_column_name(uid_column, 0, 'UID')
    set_column_name(title_column, 1, 'Title')
    set_column_name(year_column, 2, 'Year')
    set_column_name(abstract_column, 3, 'Abstract')
    set_column_name(KW_auto_column, 4, 'Auto Keywords')
    set_column_name(KW_manual_column, 4, 'Manual Keywords')
    data = {}
    for row in reader:
            year = row[year_column]
            if year not in data:
                data[year] = [row]
            else: 
                data[year].append(row) #data sorted by year
    with open('Methods_2.csv', 'w', newline='') as target_file: 
        writer = csv.writer(target_file)
        method_list = ['participat', 'creative', 'qualitative', 'quantitative', 'backcasting', 'brainstorming',  'citizen panel', 'conference', 'workshop', 'essay', 'expert panel', 'genius forecast', 'interview', 'literature review', 'morphological analysis',  'relevance tree', 'logic chart', 'logic diagram', 'role play', 'acting', 'scanning' , 'scenario workshop', 'science fictioning', 'simulation gaming', 'surveys', 'swot analysis', 'wildcard', 'benchmarking', 'bibliometrics', 'indicator',  'time series',  'modelling', 'patent analysis', 'extrapolation',  'cross-impact analysis',  'structural analysis', 'delphi', 'key technolog', 'critical technolog', 'multi-criteria', 'polling', 'voting', 'quantitative scenario', 'SMIC', 'roadmapping', 'stakeholder analysis', 'scenario', 'futures workshop']
        header_w = ['Year', '# Publications with Data']+method_list
        writer.writerow(header_w)
        for year in data:
            counter = {}
            for method in method_list:
                counter[method] = 0
                counter_pubs = 0
                for row in data[year]:
                    search_space = row[title_column]+row[abstract_column]+row[KW_auto_column]+row[KW_manual_column]
                   # print(search_space)
                    if method in search_space.lower():
                        counter[method] += 1
                    if len(search_space) > 0:
                        counter_pubs += 1
            writer.writerow([year, counter_pubs]+[counter[method] for method in method_list])
                       
print('Done')
    